# Теоретические вопросы
## Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?
**Ответ:** 
1. **micro** - используется для оценки метрик в несбалансироанных данных в которых не важна разница весов классов. 
2. **macro** - простое арифмитическое среднее между f1-оценками разных классов. Общая оценка модели.
3. **weighted** - расчет метрики с учетом весов каждого класса (их численности). Оценка несбалансированной модели в которой важна разница весов классов.

## В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?
**Ответ:** 
1. **xgboost** - давольно старый алгоритм, работает медненно, но точно. Не работает с категориальными данными.
2. **lightgbm** - новая модель, способная принимать категориальные данные, но только в числовом формате. Работает быстро, но не идеально.
3. **catboost** - новейшая модель, способна принимать категориальные данные, как в числовом, так и в строковом формате. Работает давольно быстро, если правильно настроена, результаты сравнимы с xgboost.

# Построение моделей классификации

### Описание датасета 
**подпункты с описанием добавленных столбцов после подготовки**


* **Home Ownership** - домовладение
        
        1. разложен на дамми переменные
    
* **Annual Income** - годовой доход
        
        1. Annual Income upd.2 - пропуски заполены мединой
        2. Annual Income upd.3 - пропуски заполнены моделью (близжайшие соседи)
        
* **Years in current job** - количество лет на текущем месте работы

        1. upd2 - замененные на числовые значения и заполненные модой
        2. дамми переменные заполненные модой
        3. upd.3 - пропуски заполнены моделью (близжайшие соседи)
        
* **Tax Liens** - налоговые обременения
* **Number of Open Accounts** - количество открытых счетов
* **Years of Credit History** - количество лет кредитной истории
* **Maximum Open Credit** - наибольший открытый кредит
* **Number of Credit Problems** - количество проблем с кредитом
* **Months since last delinquent** - количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротства
        
        1. upd.2 - пропуски заполены модой
        2. upd.3 - пропуски заполнены моделью (близжайшие соседи)
        
* **Purpose** - цель кредита
        
        1. разложен на дамми переменные
  
* **Term** - срок кредита
        
        1. разложен на дамми переменные
  
* **Current Loan Amount** - текущая сумма кредита
        
        1. ver.2 - выбросы заполены регрессией
        
* **Current Credit Balance** - текущий кредитный баланс
* **Monthly Debt** - ежемесячный долг
* **Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)
* **Credit Score** - кредитные очки
        
        1. ver.2 - убраны выбросы и заполнены пропуски с помощью модели
        2. ver.3 - убраны выбросы и заполнены пропуски медианой
        3. ver.4 - убраны выбросы и пропуски заполнены моделью (близжайшие соседи)

## Описание групп признаков

* TARGET_NAME - целевая переменная
* full_features_list - все признаки без пропусков
* num_features - числовые признаки
* cat_features_str - категориальные признаки (со стороквыми для **CatBoost**)
* cat_features_dumm_years_dummy - категориальные признаки разбитые на дамми переменные где "Years in current job" тоже разбит
* cat_features_dumm_years_one - категориальные признаки разбитые на дамми переменные где "Years in current job" заменен на числовые значения
* cat_features_str_to_int - категориальные признаки (где значения заменены на цифры для **LightGBM**)

# Подключаем библиотеки

In [240]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import pickle
import random

from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb

%matplotlib inline

In [241]:
import warnings
warnings.simplefilter('ignore')

In [242]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [243]:
PATH_COURSE_PROJECT_DATASET_TRAIN_PREP = 'course_project/course_project_train_prep.csv'
PATH_COURSE_PROJECT_DATASET_TRAIN = 'course_project/course_project_train.csv'

PATH_COURSE_PROJECT_DATASET_TEST_PREP = 'course_project/course_project_test_prep.csv'
PATH_COURSE_PROJECT_DATASET_TEST = 'course_project/course_project_test.csv'

SCALER_FILE_PATH = 'scaler.pkl'

TRAIN_FULL_PATH = 'course_project_train_full.csv'
TRAIN_PART_PATH = 'course_project_train_part.csv'
TEST_PART_PATH = 'course_project_test_part.csv'

# Загружаем базы данных
Чистим от не нужных столбцов 

In [244]:
df_base = pd.read_csv(PATH_COURSE_PROJECT_DATASET_TRAIN)

df_new = pd.read_csv(PATH_COURSE_PROJECT_DATASET_TRAIN_PREP)
df_new.drop('Unnamed: 0', axis=1, inplace=True)

df_test_base = pd.read_csv(PATH_COURSE_PROJECT_DATASET_TEST)
df_test_new = pd.read_csv(PATH_COURSE_PROJECT_DATASET_TEST_PREP)
df_test_new.drop('Unnamed: 0', axis=1, inplace=True)

In [245]:
df_new['NEW_2_C-P'] = df_new['NEW_2_C-P'].astype(int)

## Преобразуем строковые значения категориальных признаков в числовые для модели LGBM

In [246]:
dataset_list = [df_new, df_test_new]
for cat in df_new.select_dtypes(include='object').columns.drop('Years in current job'):
    dict_of_values_name = dict()
    repeat_number = 0
    for dataset in dataset_list:
        n = 0
        dataset[f'{cat}_for_LGBM'] = dataset[cat].copy()
        for value_name in dataset[f'{cat}_for_LGBM'].value_counts().index:
            if repeat_number == 0:
                dataset[f'{cat}_for_LGBM'].loc[dataset[f'{cat}_for_LGBM'] == value_name] = n
                dict_of_values_name[value_name] = n
                n += 1
            if repeat_number == 1:
                try:
                    dataset[f'{cat}_for_LGBM'].loc[dataset[f'{cat}_for_LGBM'] == value_name] = dict_of_values_name[value_name]
                except Exception as e:
                    dataset[f'{cat}_for_LGBM'].loc[dataset[f'{cat}_for_LGBM'] == value_name] = dict_of_values_name['other']  
        dataset[f'{cat}_for_LGBM'] = dataset[f'{cat}_for_LGBM'].astype('int64')
    repeat_number +=1

## Округляем значения Credit Score ver.2 для возможного использования признака как категориального

In [247]:
df_new['Credit Score ver.2'] = round(df_new['Credit Score ver.2'])

In [248]:
# Длинна позволяет (до 255)
len(df_new['Credit Score ver.2'].value_counts().index)

158

## Проверяем соответствие тествого и тренеровочного датасетов по признакам

In [249]:
new_fearures_list = df_test_new.columns.tolist()

In [250]:
new_fearures_list = df_test_new.columns.tolist()
for i in df_new.columns.tolist():
    try:
        new_fearures_list.remove(i)
    except:
        print(i)

new_fearures_list = df_new.columns.tolist()
for i in df_test_new.columns.tolist():
    try:
        new_fearures_list.remove(i)
    except:
        print(i)

Credit Default


## Распределяем признаки по моделям

In [251]:
TARGET_NAME = 'Credit Default'

In [252]:
full_features_list = [col for col in df_new.columns.tolist() if df_new[col].isnull().sum() == 0]

In [253]:
# тест без признака number open accounts
full_features_list.remove('Number of Open Accounts')

In [254]:
num_features = df_new[full_features_list].select_dtypes(include=['float']).columns.tolist()

In [255]:
for i in ['pca_1_H-O_OH_R', 'pca_2_H-O_OH_R', 'Credit Score ver.3', 'Annual Income upd.2', 'Current Loan Amount']:
    num_features.remove(i)
    
for i in ['Credit Score ver.2', 'Bankruptcies upd.2']:
    num_features.remove(i)


In [256]:
num_features

['Tax Liens',
 'Years of Credit History',
 'Maximum Open Credit',
 'Number of Credit Problems',
 'Current Credit Balance',
 'Monthly Debt',
 'Current Loan Amount ver.2',
 'Annual Income upd.3',
 'Credit Score ver.4',
 'Bankruptcies upd.3',
 'Years in current job upd.3']

In [257]:
cat_features_str = df_new[full_features_list].select_dtypes(include=['object', 'category']).columns.tolist()
cat_features_str.append('NEW_1_H-O')
cat_features_str.remove('Home Ownership')
cat_features_str.remove('Years in current job')
cat_features_str

['Purpose', 'Term', 'NEW_1_H-O']

In [258]:
cat_features_dumm_years_dummy = df_new[full_features_list].drop(TARGET_NAME, axis=1).select_dtypes(include=['int']).columns.tolist()[:-3]
cat_features_dumm_years_one = cat_features_dumm_years_dummy[:-10]
cat_features_dumm_years_dummy.remove('Years in current job upd.2')


In [259]:
cat_features_str_to_int = full_features_list[-3:]
cat_features_str_to_int.append('Years in current job upd.2')

In [260]:
all_categorical_features = cat_features_str_to_int + cat_features_str + cat_features_dumm_years_dummy

In [261]:
ready_features_list = num_features + all_categorical_features

In [262]:
cat_features_str_to_int

['Home Ownership_for_LGBM',
 'Purpose_for_LGBM',
 'Term_for_LGBM',
 'Years in current job upd.2']

## Приведение типов для модели  CatBoost

In [263]:
for colname in cat_features_str:
    df_new[colname] = pd.Categorical(df_new[colname])
    
df_new[cat_features_str].dtypes

catB_features = num_features + cat_features_str

## Нормализация данных

In [264]:
num_features2 = num_features.copy()
# num_features2.remove('Credit Score ver.2')

scaler = StandardScaler()

df_norm = df_new.copy()
df_norm[num_features2] = scaler.fit_transform(df_norm[num_features2])

df_new = df_norm.copy()

**Сохранение модели для нормализации данных**

In [265]:
with open(SCALER_FILE_PATH, 'wb') as file:
    pickle.dump(scaler, file)

### Разбиение на train и test

In [266]:
# X = df_new[ready_features_list]
# y = df_new[TARGET_NAME]

# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.30)

In [267]:
# X_train.shape

In [268]:
# X_test.shape

In [269]:
catB_features = list(set(catB_features))
all_categorical_features = list(set(all_categorical_features))

In [270]:
ready_features_list = list(set(ready_features_list))

## Балансировка данных

In [271]:
sm = SMOTENC(categorical_features=[df_new[ready_features_list].columns.get_loc(col) for col in all_categorical_features], n_jobs=-1)

In [272]:
X = df_new[ready_features_list]
y = df_new[TARGET_NAME]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.30)

In [273]:
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

print(f"Before OverSampling, counts of label '1': {sum(y_train==1)}")
print(f"Before OverSampling, counts of label '0': {sum(y_train==0)} \n")
print(f'After OverSampling, the shape of X_train: {X_train_res.shape}')
print(f'After OverSampling, the shape of y_train: {y_train_res.shape} \n')
print(f"After OverSampling, counts of label '1': {sum(y_train_res==1)}")
print(f"After OverSampling, counts of label '0': {sum(y_train_res==0)}")

Before OverSampling, counts of label '1': 1467
Before OverSampling, counts of label '0': 3783 

After OverSampling, the shape of X_train: (7566, 47)
After OverSampling, the shape of y_train: (7566,) 

After OverSampling, counts of label '1': 3783
After OverSampling, counts of label '0': 3783


## Сохранение обучающего и тестового датасета

In [274]:
train = pd.concat([X_train_res, y_train_res], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [275]:
df_new.to_csv(TRAIN_FULL_PATH, index=False, encoding='utf-8')
train.to_csv(TRAIN_PART_PATH, index=False, encoding='utf-8')
test.to_csv(TEST_PART_PATH, index=False, encoding='utf-8')

## Построение и оценка бозовых моделей

### Логистическая регрессия

In [276]:
model_lr = LogisticRegression()
model_lr.fit(X_train_res[cat_features_dumm_years_one + num_features], y_train_res)

y_train_pred = model_lr.predict(X_train_res[cat_features_dumm_years_one + num_features])
y_test_pred = model_lr.predict(X_test[cat_features_dumm_years_one + num_features])

get_classification_report(y_train_res, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.61      0.69      0.65      3783
           1       0.64      0.56      0.60      3783

    accuracy                           0.62      7566
   macro avg       0.63      0.62      0.62      7566
weighted avg       0.63      0.62      0.62      7566

TEST

              precision    recall  f1-score   support

           0       0.78      0.69      0.73      1604
           1       0.40      0.52      0.45       646

    accuracy                           0.64      2250
   macro avg       0.59      0.60      0.59      2250
weighted avg       0.67      0.64      0.65      2250

CONFUSION MATRIX

col_0              0    1
Credit Default           
0               1107  497
1                311  335


### KNeighbors

In [277]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train_res[cat_features_dumm_years_one + num_features], y_train_res)

y_train_pred = model_knn.predict(X_train_res[cat_features_dumm_years_one + num_features])
y_test_pred = model_knn.predict(X_test[cat_features_dumm_years_one + num_features])

get_classification_report(y_train_res, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.88      0.73      0.80      3783
           1       0.77      0.90      0.83      3783

    accuracy                           0.82      7566
   macro avg       0.83      0.82      0.82      7566
weighted avg       0.83      0.82      0.82      7566

TEST

              precision    recall  f1-score   support

           0       0.75      0.61      0.67      1604
           1       0.33      0.49      0.39       646

    accuracy                           0.57      2250
   macro avg       0.54      0.55      0.53      2250
weighted avg       0.63      0.57      0.59      2250

CONFUSION MATRIX

col_0             0    1
Credit Default          
0               972  632
1               332  314


### XGBoost

In [278]:
model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_train_res[cat_features_dumm_years_one + num_features], y_train_res)

y_train_pred = model_xgb.predict(X_train_res[cat_features_dumm_years_one + num_features])
y_test_pred = model_knn.predict(X_test[cat_features_dumm_years_one + num_features])

get_classification_report(y_train_res, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      3783
           1       0.96      0.95      0.95      3783

    accuracy                           0.95      7566
   macro avg       0.95      0.95      0.95      7566
weighted avg       0.95      0.95      0.95      7566

TEST

              precision    recall  f1-score   support

           0       0.75      0.61      0.67      1604
           1       0.33      0.49      0.39       646

    accuracy                           0.57      2250
   macro avg       0.54      0.55      0.53      2250
weighted avg       0.63      0.57      0.59      2250

CONFUSION MATRIX

col_0             0    1
Credit Default          
0               972  632
1               332  314


### LightGBM

In [279]:
model_lgbm = lgbm.LGBMClassifier()
model_lgbm.fit(X_train_res[num_features + cat_features_str_to_int], y_train_res, categorical_feature=[df_new[num_features + cat_features_str_to_int].columns.get_loc(col) for col in cat_features_str_to_int])

y_train_pred = model_lgbm.predict(X_train_res[num_features + cat_features_str_to_int])
y_test_pred = model_lgbm.predict(X_test[num_features + cat_features_str_to_int])

get_classification_report(y_train_res, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.88      0.88      0.88      3783
           1       0.88      0.88      0.88      3783

    accuracy                           0.88      7566
   macro avg       0.88      0.88      0.88      7566
weighted avg       0.88      0.88      0.88      7566

TEST

              precision    recall  f1-score   support

           0       0.76      0.75      0.76      1604
           1       0.41      0.42      0.41       646

    accuracy                           0.66      2250
   macro avg       0.58      0.59      0.59      2250
weighted avg       0.66      0.66      0.66      2250

CONFUSION MATRIX

col_0              0    1
Credit Default           
0               1205  399
1                374  272


### CatBoost

In [280]:
importance_features = num_features + cat_features_str

In [281]:
%%time

model_catb = catb.CatBoostClassifier(silent=True, 
                                     n_estimators=500,
#                                      one_hot_max_size=10, 
#                                      learning_rate=0.5, 
                                     l2_leaf_reg=10, 
#                                      iterations=700,
                                     depth=3,
#                                      class_weights=[1, 1.8],
                                     cat_features=cat_features_str)

model_catb.fit(X_train_res[importance_features], y_train_res, cat_features=cat_features_str)

y_train_pred = model_catb.predict(X_train_res[importance_features])
y_test_pred = model_catb.predict(X_test[importance_features])

get_classification_report(y_train_res, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.74      0.64      0.69      3783
           1       0.68      0.78      0.73      3783

    accuracy                           0.71      7566
   macro avg       0.71      0.71      0.71      7566
weighted avg       0.71      0.71      0.71      7566

TEST

              precision    recall  f1-score   support

           0       0.79      0.62      0.70      1604
           1       0.39      0.60      0.47       646

    accuracy                           0.62      2250
   macro avg       0.59      0.61      0.58      2250
weighted avg       0.68      0.62      0.63      2250

CONFUSION MATRIX

col_0             0    1
Credit Default          
0               999  605
1               260  386
CPU times: user 5.99 s, sys: 775 ms, total: 6.77 s
Wall time: 3.44 s


In [282]:
pd.Series(model_catb.feature_importances_, index=model_catb.feature_names_).sort_values(0)

Bankruptcies upd.3             0.277631
Tax Liens                      1.002112
Number of Credit Problems      1.818746
NEW_1_H-O                      2.515565
Purpose                        2.702066
Current Credit Balance         3.600171
Current Loan Amount ver.2      4.545931
Years of Credit History        4.703161
Maximum Open Credit            4.703268
Monthly Debt                   5.495501
Term                           6.360676
Years in current job upd.3    10.911732
Annual Income upd.3           12.337932
Credit Score ver.4            39.025508
dtype: float64